In [1]:
# imports
import pandas as pd
#from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima.arima import auto_arima

pd.options.display.max_columns = None

In [2]:
# read data
dtype = {'ID_LAT_LON_YEAR_WEEK':'string',
         'latitude': 'string',
         'longitude': 'string',
         'year': 'int',
         'week_no': 'int',
         'emission': 'float'}
df = pd.read_csv('files/train.csv', dtype=dtype)
df_test = pd.read_csv('files/test.csv', dtype=dtype)
# df

In [3]:
# prepare data
df['id'] = df['ID_LAT_LON_YEAR_WEEK'].str[:16]
day_of_week = {2019:2, 2020:3, 2021:5, 2022:6, 2023:0}
df.loc[:, 'day_of_week'] = df['year'].map(day_of_week)
df.loc[:, 'date'] = df['year'].astype('string') + '-' + df['week_no'].astype('string') + '-' + df['day_of_week'].astype('string')
df.loc[:, 'date'] = pd.to_datetime(df['date'], format='%Y-%W-%w')

df_test.insert(0, 'id', df_test['ID_LAT_LON_YEAR_WEEK'].str[:16])
# df

In [4]:
endog_groups = df[['id', 'date', 'emission']].groupby('id')

exog = df.drop(columns=['ID_LAT_LON_YEAR_WEEK', 'latitude', 'longitude', 'year', 'week_no', 'emission', 'day_of_week'])
exog.columns = list(range(70)) + ['id', 'date']
exog.insert(0, 'date', exog.pop('date'))
exog.insert(0, 'id', exog.pop('id'))
exog_groups = exog.groupby('id')

df_test = df_test.sort_values(['id', 'year', 'week_no']).drop(columns=['ID_LAT_LON_YEAR_WEEK', 'latitude', 'longitude', 'year', 'week_no'])
df_test.columns = ['id'] + list(range(70))
test_groups = df_test.groupby('id')

In [21]:
# function to create model and forecast
def autosarimax(endog, exog, exog_test, steps=49):
    if len(exog.columns) < 1:
        exog = None
        exog_test = None
    else:
        exog = exog.values
        exog_test = exog_test.values
    autosarimax = auto_arima(y = endog.loc[:, 'emission'].values,
                             X = exog)
    autosarimax = autosarimax.fit(y = endog.loc[:, 'emission'].values,
                                  X = exog,
                                  full_output = False,
                                  disp = False)
    return autosarimax.predict(n_periods = steps,
                               X = exog_test)

In [22]:
%%time
# run sarimax function for every location
results = {}

for i, (name, endog_group) in enumerate(endog_groups):
    if i%10 == 0: print(f'{int(100*i/endog_groups.ngroups)} %')
    df_endog_id = endog_group.drop(columns='id').set_index('date', drop=True).resample('W').nearest()
    df_exog_id = exog_groups.get_group(name).drop(columns='id').set_index('date', drop=True).resample('W').nearest()
    df_exog_id = df_exog_id.loc[:, df_exog_id.isna().mean() < .1].interpolate().bfill().ffill()
    df_exog_id_test = test_groups.get_group(name).drop(columns='id').interpolate().bfill().ffill()#.set_index('date', drop=True).resample('W').nearest()
    df_exog_id_test = df_exog_id_test[df_exog_id.columns]
    test = autosarimax(df_endog_id, df_exog_id, df_exog_id_test)
    results[name] = test
df_results = pd.DataFrame(results).T.reset_index()
df_results

0 %


C:\Users\Andrew\miniconda3\envs\kaggle\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1899: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1
C:\Users\Andrew\miniconda3\envs\kaggle\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1906: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1
C:\Users\Andrew\miniconda3\envs\kaggle\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1899: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1
C:\Users\Andrew\miniconda3\envs\kaggle\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1906: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1
C:\Users\Andrew\miniconda3\envs\kaggle\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1899: RuntimeWarning: divide by zero encountered in reciprocal
  return n

2 %


C:\Users\Andrew\miniconda3\envs\kaggle\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1899: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1
C:\Users\Andrew\miniconda3\envs\kaggle\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1906: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1
C:\Users\Andrew\miniconda3\envs\kaggle\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1906: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1
C:\Users\Andrew\miniconda3\envs\kaggle\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1906: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1
C:\Users\Andrew\miniconda3\envs\kaggle\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1899: RuntimeWarning: divide by zero encountered in reciprocal
  return n

4 %


...

C:\Users\Andrew\miniconda3\envs\kaggle\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1899: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1
C:\Users\Andrew\miniconda3\envs\kaggle\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1906: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1


98 %


C:\Users\Andrew\miniconda3\envs\kaggle\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1899: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1
C:\Users\Andrew\miniconda3\envs\kaggle\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1899: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1


CPU times: total: 3h 10min 39s
Wall time: 3h 8min 59s


,index,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48
0,ID_-0.510_29.290,3.578441,3.998130,4.177185,4.167426,4.380077,4.410263,4.346310,4.093629,4.018802,4.302843,4.270852,4.317106,4.117228,3.856638,3.797460,4.285344,5.139314,3.644323,3.965504,4.320294,4.116161,4.046932,3.939018,4.446988,4.514902,4.606638,4.962333,4.981032,4.635417,4.979407,5.167239,4.699088,5.294084,5.135377,5.129022,4.749328,5.112852,4.990209,4.787478,5.120494,4.774278,4.963072,5.188317,5.451506,5.757096,5.462516,-2.998590,4.611031,4.720119
1,ID_-0.528_29.472,3.984804,4.182931,3.794387,4.468849,4.963955,4.309669,4.813111,4.477295,5.163820,4.848080,4.101785,4.858835,3.658950,4.699039,3.976310,4.252321,3.998786,4.498422,4.195237,3.635043,4.102757,3.811959,3.845172,3.599255,4.578391,3.948689,4.272830,4.549452,4.040163,3.710309,3.077774,4.523870,4.490255,4.771867,4.435570,4.787281,4.782286,4.563611,4.563109,5.284353,4.304752,4.820544,5.287675,4.846929,5.019035,4.928563,-1.015974,5.041680,5.299891
2,ID_-0.547_29.653,0.511972,0.522355,0.596686,0.613425,0.674077,0.661358,0.653126,0.626441,0.667443,0.673822,0.581000,0.664908,0.578348,0.587944,0.586660,0.648593,0.574181,0.529844,0.618828,0.591288,0.627872,0.612178,0.646064,0.665382,0.690114,0.724271,0.695628,0.664236,0.708578,0.683949,0.700587,0.668320,0.650910,0.693772,0.660001,0.708893,0.696209,0.739766,0.768852,0.699776,0.717498,0.734328,0.708342,0.755311,0.685724,0.683315,1.172501,0.700853,0.644634
3,ID_-0.569_30.031,110.993610,112.517735,117.647625,111.135845,117.674962,112.308767,110.826259,122.380302,105.473930,117.228529,113.454748,119.181186,112.503494,114.803470,116.384686,119.260097,110.304972,100.065514,115.141258,115.049151,117.464509,109.755863,127.455135,119.057856,127.195444,120.305025,122.024458,131.050152,123.853616,118.914798,130.665657,121.659446,133.195601,130.322383,126.951907,138.604985,140.360496,140.071214,143.224885,154.730556,148.419830,148.638001,145.598175,154.136604,151.185232,147.363784,136.829919,141.576474,146.710939
4,ID_-0.598_29.102,0.085480,0.087547,0.092392,0.081485,0.085885,0.080062,0.085516,0.083156,0.080923,0.098265,0.088652,0.098952,0.091285,0.078337,0.087735,0.090364,0.087645,0.082953,0.090983,0.090923,0.095009,0.096220,0.102040,0.104958,0.104775,0.102463,0.099878,0.108457,0.098986,0.106849,0.100124,0.107756,0.106350,0.101584,0.103478,0.110388,0.102518,0.105044,0.110086,0.104918,0.115314,0.120376,0.103367,0.117484,0.114189,0.115053,0.475769,0.113678,0.102089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,ID_-3.153_30.347,14.771227,16.417780,16.522413,14.349629,15.149342,13.634964,12.820321,14.846805,13.269030,14.953678,13.608090,14.520034,13.802129,13.907957,13.561242,12.105701,12.117798,12.390417,12.846058,13.406666,12.350827,13.192372,13.684537,13.126844,13.685913,12.900316,16.084691,14.948918,15.346694,13.773515,14.792646,16.003258,13.959385,16.930184,14.938079,15.141150,15.825469,16.356388,17.652048,17.024587,18.456084,19.171887,17.232205,18.666779,17.113953,17.694159,-1984.982464,16.968286,16.516575
493,ID_-3.161_28.839,0.122746,0.124169,0.133911,0.125606,0.121393,0.119776,0.118752,0.119776,0.112136,0.127682,0.111871,0.116565,0.114317,0.107521,0.118175,0.122522,0.116376,0.107331,0.115974,0.106755,0.115561,0.123790,0.128731,0.136090,0.130154,0.129623,0.138746,0.145101,0.136574,0.138178,0.149881,0.144067,0.156096,0.142771,0.144937,0.145111,0.152352,0.151168,0.152128,0.151736,0.152105,0.156451,0.150387,0.163317,0.149865,0.148870,0.108588,0.147977,0.140031
494,ID_-3.174_29.926,42.346913,42.285981,45.841601,40.493576,43.224044,41.960316,43.874106,40.126861,43.654071,45.980622,43.661529,43.890059,40.102109,37.902373,42.963367,41.098491,41.240688,42.955750,46.424945,43.797534,44.272548,42.186211,44.575229,46.879297,45.128498,44.61

In [23]:
# format output
df_results = df_results.melt(id_vars=['index'], value_vars=range(49))
df_results['id'] = df_results['index'] + '_2022_' + df_results['variable'].astype('string').str.zfill(2)
df_results = df_results[['id', 'value']]
df_results = df_results.sort_values('id').reset_index(drop=True)
df_results.columns = ['ID_LAT_LON_YEAR_WEEK', 'emission']
df_results

,ID_LAT_LON_YEAR_WEEK,emission
0,ID_-0.510_29.290_2022_00,3.578441
1,ID_-0.510_29.290_2022_01,3.998130
2,ID_-0.510_29.290_2022_02,4.177185
3,ID_-0.510_29.290_2022_03,4.167426
4,ID_-0.510_29.290_2022_04,4.380077
...,...,...
24348,ID_-3.299_30.301_2022_44,33.825590
24349,ID_-3.299_30.301_2022_45,32.572584
24350,ID_-3.299_30.301_2022_46,-750.872112
24351,ID_-3.299_30.301_2022_47,32.761105


In [24]:
# save output to csv
df_results.to_csv('output/autosarimax.csv', index=False)